In [ ]:
!pip install cvxpy
!pip install sympy SumOfSquares

In this notebook, we search for a Lyapunov function $V(\boldsymbol{x})$ that certifies the stability of the nonlinear system:

$$
\begin{align*}
  \dot{x}_1 &= -x_1 - 2 x_2^2 ,  \\
  \dot{x}_2 &= -x_2 - x_1 x_2 - 2 x_2^3 .
\end{align*}
$$

We parameterize the Lyapunov function as

$$
V(\boldsymbol{x}) = \frac{1}{2} \boldsymbol{x}^\top \underbrace{\begin{bmatrix} p_{11} & p_{21} \\ p_{21} & p_{22} \end{bmatrix}}_{\mathbf{P}} \boldsymbol{x} .
$$

The time-derivative of the Lyapunov function is

$$
\dot{V}(\boldsymbol{x}) = \boldsymbol{x}^\top \mathbf{P} \dot{\boldsymbol{x}} .
$$

For Lyapunov stability, $V(\boldsymbol{x})$ must be positive, and $\dot{V}(\boldsymbol{x})$ must be negative.

### Using semi-definite programming (SDP)

In [1]:
import cvxpy as cp

P = cp.Variable((2, 2), symmetric=True)
Q = cp.Variable((4, 4), symmetric=True)

constraints = [
    P >> 0,
    Q >> 0,
    Q[0,0] == P[0,0],
    Q[1,0] == P[1,0],
    Q[1,1] == P[1,1],
    Q[2,0] == 0.5 * P[1,0],
    Q[2,1] + Q[3,0] == P[0,0] + 0.5 * P[1,1],
    Q[2,2] == 0,
    Q[3,1] == P[1,0],
    Q[3,2] == 2 * P[1,0],
    Q[3,3] == 2 * P[1,1],
    P[0,0] == 1, # fix p_11 to 1
]

prob = cp.Problem(cp.Minimize(0), constraints)
prob.solve(solver=cp.CVXOPT)

print("Status: ", prob.status)
print("P:\n", P.value)

Status:  optimal
P:
 [[1.00000000e+00 8.37553130e-22]
 [8.37553130e-22 2.00010433e+00]]


### Using sum of squares (SOS) programming

In [2]:
import numpy
import sympy as sp
from SumOfSquares import SOSProblem

x1, x2 = sp.symbols('x_1 x_2')
x = sp.Matrix([[x1], [x2]])

x1_dot = -x1 - 2 * x2**2
x2_dot = -x2 - x1 * x2 - 2 * x2**3
x_dot = sp.Matrix([[x1_dot], [x2_dot]])

p11, p21, p22 = sp.symbols('p_{11} p_{21} p_{22}')
P = sp.Matrix(
    [[p11, p21],
     [p21, p22]])

V = 0.5 * x.T @ P @ x
V_dot = x.T @ P @ x_dot

prob = SOSProblem()
prob.add_sos_constraint(V[0], [x1, x2])
prob.add_sos_constraint(-V_dot[0], [x1, x2])
prob.add_constraint(prob.sym_to_var(p11) == 1) # fix p_11 to 1

prob.solve(solver='cvxopt')

print("Status: ", prob.status)
print("P:\n", numpy.array(
    [[prob.sym_to_var(p11).value, prob.sym_to_var(p21).value],
     [prob.sym_to_var(p21).value, prob.sym_to_var(p22).value]]))

Status:  optimal
P:
 [[ 1.00000000e+00 -5.49550564e-07]
 [-5.49550564e-07  1.99161535e+00]]
